In [3]:
import pickle as pkl
import numpy as np
"""
for dataset in ['iwslt14', 'multi30k']:
    logs = pkl.load(open('data/{dataset}_logs.pkl'.format(dataset=dataset), 'rb'))

    num_datapoints = len(logs['sequences'])

    iterations = logs['normal_A'].keys()
    dicts = logs['sequences']
    for key in logs:
        if key == 'sequences':
            continue
        for iteration in iterations:
            for datapoint_idx in range(num_datapoints):
                dicts[datapoint_idx][(key, iteration, 'alpha')] = logs[key][iteration][datapoint_idx]['alpha']
                dicts[datapoint_idx][(key, iteration, 'beta')] = logs[key][iteration][datapoint_idx]['beta']
                dicts[datapoint_idx]['split'] = logs[key][iteration][datapoint_idx]['split']
    pkl.dump(dicts, open('data/{dataset}_logs_rz.pkl'.format(dataset=dataset), 'wb'))
    """
from utils import *

def max_acc_iter(name, metas):
    max_acc = -1.0
    max_acc_iter = None
    for key_ in metas:
        if key_[0] == name and 'test' in key_[2]:
            if metas[key_] > max_acc:
                max_acc = metas[key_]
                max_acc_iter = key_[1]
    return max_acc_iter

In [34]:
dataset = 'Furd'
dat = pkl.load(open('outputs/{dataset}_logs.pkl'.format(dataset=dataset), 'rb'))
all_dicts = dat['data']
subset = 'val'
dicts = [d for d in all_dicts if d['split'] == subset]
iterations = sorted(list(set([key[1] for key in dat['metas']])))

In [35]:
from utils import TopKMatch, TopPercentMatch, SpearmanRankCorr
metrics = [TopKMatch(k=1), TopKMatch(k=3), TopKMatch(k=5), TopPercentMatch(p=5)]
normalA_iter = max_acc_iter('normal_A', dat['metas'])
normalB_iter = max_acc_iter('normal_B', dat['metas'])
uniform_iter = max_acc_iter('uniform', dat['metas'])

In [36]:
# metrics.append(SpearmanRankCorr()) comment out because this takes a lot of time to evaluate
key1, key2 = ('normal_A', normalA_iter, 'alpha'), ('normal_B', normalB_iter, 'beta')
print(key1, key2)
for metric in metrics:
    print(metric.eval_corr(dicts, key1, key2))

('normal_A', 3000, 'alpha') ('normal_B', 3999, 'beta')
{'name': 'top 1 match', 'correlation': 0.059113300492610835, 'baseline': 0.07600281491907107}
{'name': 'top 3 match', 'correlation': 0.16819141449683322, 'baseline': 0.21463757916959889}
{'name': 'top 5 match', 'correlation': 0.2660098522167488, 'baseline': 0.32371569317382126}
{'name': 'top 5% match', 'correlation': 0.0752990851513019, 'baseline': 0.0781140042223786}


In [37]:
key1, key2 = ('normal_A', normalA_iter, 'alpha'), ('uniform', uniform_iter, 'beta')
print(key1, key2)
for metric in metrics:
    print(metric.eval_corr(dicts, key1, key2))

('normal_A', 3000, 'alpha') ('uniform', 3000, 'beta')
{'name': 'top 1 match', 'correlation': 0.03518648838845883, 'baseline': 0.07178043631245602}
{'name': 'top 3 match', 'correlation': 0.12526389866291343, 'baseline': 0.199155524278677}
{'name': 'top 5 match', 'correlation': 0.25193525686136525, 'baseline': 0.3286418015482055}
{'name': 'top 5% match', 'correlation': 0.045038705137227304, 'baseline': 0.09429978888106967}


In [38]:
key1, key2 = ('uniform', uniform_iter, 'beta'), 'logistics_beta'
print(key1, key2)
for metric in metrics:
    print(metric.eval_corr(dicts, key1, key2))

('uniform', 3000, 'beta') logistics_beta
{'name': 'top 1 match', 'correlation': 0.09078114004222379, 'baseline': 0.08515130190007038}
{'name': 'top 3 match', 'correlation': 0.2674173117522871, 'baseline': 0.21604503870513722}
{'name': 'top 5 match', 'correlation': 0.4560168895144265, 'baseline': 0.3645320197044335}
{'name': 'top 5% match', 'correlation': 0.11611541168191415, 'baseline': 0.09218859957776214}
